In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\kidney-disease-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
   root_dir: Path
   model_checkpoint: Path
   updated_base_model_path:Path
   training_data:Path
   params_epoch:int
   params_batch_size:int
   params_image_size:list
   params_augmentation:bool

In [5]:
from cnnClassifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        
        create_directories([self.config.artifacts_root])
        
        
    def get_training_config(self) -> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")
        
        create_directories([training.root_dir])
        
        training_config = TrainingConfig(
            root_dir=training.root_dir,
            model_checkpoint=training.model_checkpoint,
            updated_base_model_path=prepare_base_model.updated_base_model_path,
            training_data=training_data,
            params_epoch=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_augmentation=params.AUGMENTATION
        )

        return training_config

In [7]:
import os
from zipfile import ZipFile
import urllib.request as request
import tensorflow as tf
import time


In [12]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    
    def train_valid_generator(self):

        ## data generator copied from keras documentation
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.2
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
    

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
            )
            
        else:
            train_datagenerator=valid_datagenerator
            
        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)


    def train(self):
        # formula used for calculating steps per epoch and validation steps
        self.steps_per_epochs = self.train_generator.samples // self.config.params_batch_size
        self.validation_steps = self.valid_generator.samples // self.config.params_batch_size
        
        self.model.fit(
            self.train_generator,
            validation_data=self.valid_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epochs,
            validation_steps=self.validation_steps
        )

        self.save_model(path=self.config.model_checkpoint, model=self.model)

In [13]:
try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    print(f"Error occurred: {e}")

YAML content loaded: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharin', 'local_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/updated_model.h5'}, 'training': {'root_dir': 'artifacts/training', 'model_checkpoint': 'artifacts/training/model.h5'}}
YAML content loaded: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 32, 'INCLUDE_TOP': False, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'EPOCHS': 200, 'LEARNING_RATE': 0.01}
[2025-08-19 20:22:14,035] INFO in common: Created directory: artifacts
[2025-08-19 20:22:14,036] INFO in common: Created directory: artifacts/training
Found 93 images belonging to 2 classes.
Found 372